In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


# Intro
In this small notebook we will classify the well know and easy Iris dataset using a new ural network implemented with PyTorch.

# Imports

In [2]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import pandas as pd

# Loading the data, minor EDA and preprocessing
In this section we will load the data and do a very quick EDA on it.

We split our dataset in X (features) and y (labels), the label is the species of the plant.
The labels will be encoded using the LabelEncoder, and the features will be scaled.

At the end of this section we split X and y to train and test end convert them to tensors.

In [3]:
df = pd.read_csv("/kaggle/input/iris/Iris.csv")

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
# the sape of the dataset
print(f"shape of the dataset: {df.shape}")

# the unique values for the Species column, which is our label
print("distinct species: ", df["Species"].unique())

shape of the dataset: (150, 6)
distinct species:  ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [5]:
# we define out X and y and use label encoder for our labels
labelEncoder = LabelEncoder()

X = df[["SepalLengthCm",  "SepalWidthCm",  "PetalLengthCm", "PetalWidthCm"]]
y = labelEncoder.fit_transform(df["Species"])

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

labelEncoder.classes_

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


# Defining the Model
Next we will define out model. It's an easy model, it consists of three fully connected layers, with ReLU activation layres between them. We use a LogSoftmx at the end of the network.

We output 3 values as this is the number of classes.

In [8]:
class IrisNNModel(nn.Module):
    def __init__(self):
        super(IrisNNModel, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(in_features=4, out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=3),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, X):
        yHat = self.net(X)

        return yHat

# Training the model
Here we will train out model, first we declare the variables we need and the we do the actual training loop.

In [9]:
model = IrisNNModel()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lossFunction = nn.CrossEntropyLoss()

BATCH_SIZE = 16

print(len(X_train), len(y_train))

datasetTrain = TensorDataset(X_train, y_train)
dataloaderTrain = DataLoader(datasetTrain, batch_size=BATCH_SIZE, shuffle=True)

model.train()

BATCH_SIZE = 16

datasetTrain = TensorDataset(X_train, y_train)
dataloaderTrain = DataLoader(datasetTrain, batch_size=BATCH_SIZE, shuffle=True)

120 120


In [10]:
for epoch in range(1000):
    for i, (X_batch, y_batch) in enumerate(dataloaderTrain):

        yHatPred = model.forward(X_batch)

        loss = lossFunction(yHatPred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i == 0 and epoch % 50 == 0:
            print(f"training epoch {epoch}, batch {i}, loss {loss.item():.8f}")

training epoch 0, batch 0, loss 1.04355180
training epoch 50, batch 0, loss 0.09736168
training epoch 100, batch 0, loss 0.02706866
training epoch 150, batch 0, loss 0.10061383
training epoch 200, batch 0, loss 0.05996988
training epoch 250, batch 0, loss 0.01234431
training epoch 300, batch 0, loss 0.02686932
training epoch 350, batch 0, loss 0.00119282
training epoch 400, batch 0, loss 0.00884304
training epoch 450, batch 0, loss 0.01071633
training epoch 500, batch 0, loss 0.00003681
training epoch 550, batch 0, loss 0.00473316
training epoch 600, batch 0, loss 0.00325835
training epoch 650, batch 0, loss 0.00141550
training epoch 700, batch 0, loss 0.00126614
training epoch 750, batch 0, loss 0.00251437
training epoch 800, batch 0, loss 0.00063520
training epoch 850, batch 0, loss 0.00000292
training epoch 900, batch 0, loss 0.00034853
training epoch 950, batch 0, loss 0.00029381


# Making predictions
First we are going to define the dataloader we need, and set our model in eval mode, and the we will do the predictions for our test data.
As we have small test dataset we will print the actual predictions for all the data.

In [11]:
model.eval()

datasetTest = TensorDataset(X_test, y_test)
dataloaderTest = DataLoader(datasetTest, batch_size=BATCH_SIZE)

In [12]:
with torch.no_grad():
    for i, (X_batch, y_batch) in enumerate(dataloaderTest):

        yHat = model.forward(X_batch)
        loss = lossFunction(yHat, y_batch)

        print(f"### prediction for batch {i}, batch loss: {loss.item():.4f}")

        for i in range(yHat.shape[0]):
            # we use argmax on the result to determine the max value, that is our prediction
            print(f"actual value: {y_batch[i].item()}", f"predicted value: {torch.argmax(yHat[i], dim=0).item()}")

### prediction for batch 0, batch loss: 0.0005
actual value: 1 predicted value: 1
actual value: 0 predicted value: 0
actual value: 2 predicted value: 2
actual value: 1 predicted value: 1
actual value: 1 predicted value: 1
actual value: 0 predicted value: 0
actual value: 1 predicted value: 1
actual value: 2 predicted value: 2
actual value: 1 predicted value: 1
actual value: 1 predicted value: 1
actual value: 2 predicted value: 2
actual value: 0 predicted value: 0
actual value: 0 predicted value: 0
actual value: 0 predicted value: 0
actual value: 0 predicted value: 0
actual value: 1 predicted value: 1
### prediction for batch 1, batch loss: 0.0000
actual value: 2 predicted value: 2
actual value: 1 predicted value: 1
actual value: 1 predicted value: 1
actual value: 2 predicted value: 2
actual value: 0 predicted value: 0
actual value: 2 predicted value: 2
actual value: 0 predicted value: 0
actual value: 2 predicted value: 2
actual value: 2 predicted value: 2
actual value: 2 predicted value

# Finish
You have made it to the finish, thank you! Please upvote :)